# Case study 1
* Re-ranking leaderboards.
* Agreement rate & Kendall-Tau correlation for comparing rankings.
* Selecting the winner.

In [1]:
import pandas as pd
import numpy as np
import sys

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

sys.path.append("..")

from votenrank import Leaderboard
from votenrank.utils import agreement_rate, kendall_tau
from votenrank.data_processing import preprocess_glue, preprocess_sglue, preprocess_value

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

loading region bounding boxes for computing carbon emissions region, this may take a moment...
 454/454... rate=577.94 Hz, eta=0:00:00, total=0:00:000
Done!


/home/data_sapiens/.local/lib/python3.8/site-packages/experiment_impact_tracker/data_interface.py:37: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


## GLUE

In [2]:
glue, glue_weights = preprocess_glue(pd.read_csv("../tables/leaderboards - GLUE.csv"), head=30)
glue.head()

/home/data_sapiens/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,CoLA,SST-2,MNLI-m,MNLI-mm,QNLI,RTE,WNLI,AX,MRPC_n1,MRPC_n2,STS-B_n1,STS-B_n2,QQP_n1,QQP_n2
Model,,,,,,,,,,,,,,
ERNIE,75.5,97.8,92.3,91.7,97.3,92.6,95.9,51.7,93.9,91.8,93.0,92.6,75.2,90.9
StructBERT + CLEVER,75.3,97.7,91.7,91.5,97.4,92.5,95.2,49.1,93.9,91.9,93.5,93.1,75.6,90.8
DeBERTa / TuringNLRv4,71.5,97.5,91.9,91.6,99.2,93.2,94.5,53.2,94.0,92.0,92.9,92.6,76.2,90.8
DeBERTa + CLEVER,73.4,97.5,92.1,91.7,96.5,92.8,96.6,35.2,92.8,90.4,93.2,92.9,76.3,90.8
MacALBERT + DKM,74.8,97.0,91.3,91.1,97.8,92.0,94.5,52.6,94.5,92.6,92.8,92.6,74.7,90.6


### Ranking

In [3]:
not_nan_glue_lb = Leaderboard(glue.dropna(), weights=glue_weights)
not_nan_glue_lb_rank = not_nan_glue_lb.rank_all()
not_nan_glue_lb_rank

,Plurality,AM,GM,Minimax,Copeland,Dowdall,Borda
Ranking position,,,,,,,
1,2.00: ERNIE,91.18: ERNIE,90.89: ERNIE,-0.00: ERNIE,29.00: ERNIE,4.95: ERNIE,260.50: ERNIE
2,2.00: GLUE Human Baselines,91.07: StructBERT + CLEVER,90.78: StructBERT + CLEVER,-5.50: DeBERTa + CLEVER,25.00: DeBERTa + CLEVER,4.08: GLUE Human Baselines,256.00: StructBERT + CLEVER
3,1.50: DeBERTa + CLEVER,90.88: DeBERTa + CLEVER,90.56: DeBERTa + CLEVER,-6.00: DeBERTa / TuringNLRv4,24.00: StructBERT + CLEVER,3.82: DeBERTa + CLEVER,247.50: DeBERTa + CLEVER
4,1.00: MacALBERT + DKM,90.86: DeBERTa / TuringNLRv4,90.48: DeBERTa / TuringNLRv4,-6.50: StructBERT + CLEVER,22.00: T5,3.41: DeBERTa / TuringNLRv4,241.50: DeBERTa / TuringNLRv4
5,1.00: StructBERT + CLEVER,90.74: MacALBERT + DKM,90.44: MacALBERT + DKM,-7.00: T5,22.00: GLUE Human Baselines,3.27: StructBERT + CLEVER,233.50: ALBERT + DAAF + NAS
6,0.50: ALBERT + DAAF + NAS,90.66: ALBERT + DAAF + NAS,90.34: ALBERT + DAAF + NAS,-7.00: GLUE Human Baselines,22.00: DeBERTa / TuringNLRv4,2.57: MacALBERT + DKM,229.50: T5
7,0.00: DeBERTa / TuringNLRv4,90.48: T5,90.11: T5,-7.00: ALBERT + DAAF + NAS,16.00: ALBERT + DAAF + NAS,2.55: T5,220.50: MacALBERT + DKM
8,0.00: ConvBERT base,90.01: NEZHA-Large,89.65: NEZHA-Large,-8.00: MacALBERT + DKM,16.00: MacALBERT + DKM,2.39: ALBERT + DAAF + NAS,206.50: NEZHA-Large
9,0.00: LV-BERT-base,89.96: MT-DNN-SMART,89.51: MT-DNN-SMART,-9.00: MT-DNN-SMART,13.00: NEZHA-Large,2.14: MT-DNN-SMART,203.00: Funnel-Transformer (Ensemble B10-10-10H1024)


### Agreement rate: top-k

In [4]:
top_k = [1, 3, 5, 7]

ar_top_k = pd.DataFrame(
    agreement_rate(not_nan_glue_lb_rank, k, top_k=True) for k in top_k
)
ar_top_k["top_k"] = top_k
ar_top_k.set_index("top_k")

,Plurality,GM,Minimax,Copeland,Dowdall,Borda
top_k,,,,,,
1,1.00,1.0,1.00,1.00,1.00,1.0
3,0.67,1.0,0.67,1.00,0.67,1.0
5,0.80,1.0,0.80,0.60,0.80,0.8
7,0.86,1.0,0.86,0.86,0.86,1.0


### Agreement rate: least-k

In [5]:
least_k = [3, 7]

ar_least_k = pd.DataFrame(
    agreement_rate(not_nan_glue_lb_rank, k, top_k=False) for k in least_k
)
ar_least_k["least_k"] = least_k
ar_least_k.set_index("least_k")

,Plurality,GM,Minimax,Copeland,Dowdall,Borda
least_k,,,,,,
3,0.33,0.67,0.33,1.0,1.0,1.0
7,0.14,0.86,0.14,1.0,1.0,1.0


### Kendall-Tau: total rankings

In [6]:
pd.DataFrame(kendall_tau(not_nan_glue_lb_rank).items(), columns=["Method", "Tau"])

,Method,Tau
0,Plurality,0.030
1,GM,0.563
2,Minimax,-0.057
3,Copeland,0.228
4,Dowdall,0.283
5,Borda,0.411


### Selecting the winner

In [7]:
not_nan_glue_lb.elect_all()

,method,winners
0,Plurality,"[ERNIE, GLUE Human Baselines]"
1,Threshold,[StructBERT + CLEVER]
2,Condorcet,[ERNIE]
3,AM,[ERNIE]
4,GM,[ERNIE]
5,Minimax,[ERNIE]
6,Copeland,[ERNIE]
7,Dowdall,[ERNIE]
8,Baldwin,[ERNIE]
9,Borda,[ERNIE]


## SuperGLUE

In [8]:
from votenrank.data_processing import preprocess_sglue

sglue, sglue_weights = preprocess_sglue(pd.read_csv("../tables/leaderboards - SuperGLUE.csv"))
sglue.head()

,BoolQ,COPA,RTE,WiC,WSC,AX-b,CB_n1,CB_n2,MultiRC_n1,MultiRC_n2,ReCoRD_n1,ReCoRD_n2,AX-g_n1,AX-g_n2
Model,,,,,,,,,,,,,,
ERNIE 3.0,91.0,97.4,92.6,77.4,97.3,68.6,98.6,99.2,88.6,63.2,94.7,94.2,92.7,94.7
"T5 + UDG, Single Model (Google Brain)",91.4,98.0,93.0,77.9,96.6,69.1,95.8,97.6,88.3,63.0,94.2,93.5,92.7,91.9
DeBERTa / TuringNLRv4,90.4,98.4,93.2,77.5,95.9,66.7,95.7,97.6,88.2,63.7,94.5,94.1,93.3,93.8
SuperGLUE Human Baselines,89.0,100.0,93.6,80.0,100.0,76.6,95.8,98.9,81.8,51.9,91.7,91.3,99.3,99.7
T5,91.2,94.8,92.5,76.9,93.8,65.6,93.9,96.8,88.1,63.3,94.1,93.4,92.7,91.9


### Ranking

In [9]:
not_nan_sglue_lb = Leaderboard(sglue.dropna(), weights=sglue_weights)
not_nan_sglue_lb_rank = not_nan_sglue_lb.rank_all()
not_nan_sglue_lb_rank

,Plurality,AM,GM,Minimax,Copeland,Dowdall,Borda
Ranking position,,,,,,,
1,4.00: SuperGLUE Human Baselines,90.62: ERNIE 3.0,90.04: ERNIE 3.0,-0.00: ERNIE 3.0,20.00: SuperGLUE Human Baselines,4.98: SuperGLUE Human Baselines,155.00: ERNIE 3.0
2,2.50: ERNIE 3.0,"90.39: T5 + UDG, Single Model (Google Brain)","89.84: T5 + UDG, Single Model (Google Brain)",-0.00: SuperGLUE Human Baselines,19.00: ERNIE 3.0,4.25: ERNIE 3.0,"154.50: T5 + UDG, Single Model (Google Brain)"
3,"1.00: T5 + UDG, Single Model (Google Brain)",90.29: DeBERTa / TuringNLRv4,89.75: DeBERTa / TuringNLRv4,"-4.50: T5 + UDG, Single Model (Google Brain)","18.00: T5 + UDG, Single Model (Google Brain)","3.62: T5 + UDG, Single Model (Google Brain)",153.00: DeBERTa / TuringNLRv4
4,0.50: DeBERTa / TuringNLRv4,89.79: SuperGLUE Human Baselines,88.80: SuperGLUE Human Baselines,-5.00: DeBERTa / TuringNLRv4,15.00: DeBERTa / TuringNLRv4,3.29: DeBERTa / TuringNLRv4,145.50: SuperGLUE Human Baselines
5,0.00: iPET (ALBERT) - Few-Shot (32 Examples),89.25: T5,88.75: T5,-7.50: T5,13.00: T5,2.11: T5,141.50: T5
6,0.00: CBoW,86.65: NEZHA-Plus,85.93: NEZHA-Plus,-8.00: iPET (ALBERT) - Few-Shot (32 Examples),11.00: NEZHA-Plus,1.16: NEZHA-Plus,116.50: NEZHA-Plus
7,0.00: Most Frequent Class,86.09: PAI Albert,85.38: PAI Albert,-8.00: CBoW,9.00: RoBERTa-iCETS,1.06: RoBERTa-iCETS,108.00: RoBERTa-iCETS
8,0.00: BERT,85.95: RoBERTa-iCETS,85.36: RoBERTa-iCETS,-8.00: Most Frequent Class,7.00: PAI Albert,1.03: PAI Albert,105.50: PAI Albert
9,0.00: BERT++,85.88: RoBERTa (ensemble),85.13: RoBERTa (ensemble),-8.00: BERT,5.00: RoBERTa-mtl-adv,0.97: RoBERTa-mtl-adv,100.50: RoBERTa (ensemble)


### Agreement rate: top-k

In [10]:
ar_top_k = pd.DataFrame(
    agreement_rate(not_nan_sglue_lb_rank, k, top_k=True) for k in top_k
)
ar_top_k["top_k"] = top_k
ar_top_k.set_index("top_k")

,Plurality,GM,Minimax,Copeland,Dowdall,Borda
top_k,,,,,,
1,0.00,1.0,1.00,0.00,0.00,1.00
3,0.67,1.0,0.67,0.67,0.67,1.00
5,0.80,1.0,1.00,1.00,1.00,1.00
7,0.57,1.0,0.71,0.86,0.86,0.86


### Agreement rate: least-k

In [11]:
ar_least_k = pd.DataFrame(
    agreement_rate(not_nan_sglue_lb_rank, k, top_k=False) for k in least_k
)
ar_least_k["least_k"] = least_k
ar_least_k.set_index("least_k")

,Plurality,GM,Minimax,Copeland,Dowdall,Borda
least_k,,,,,,
3,0.33,1.00,0.33,1.00,1.00,1.0
7,0.14,0.86,0.14,0.86,0.86,1.0


### Kendall-Tau: total rankings

In [12]:
pd.DataFrame(kendall_tau(not_nan_sglue_lb_rank).items(), columns=["Method", "Tau"])

,Method,Tau
0,Plurality,-0.152
1,GM,0.455
2,Minimax,-0.056
3,Copeland,0.082
4,Dowdall,0.126
5,Borda,0.238


### Selecting the winner

In [13]:
not_nan_sglue_lb.elect_all()

,method,winners
0,Plurality,[SuperGLUE Human Baselines]
1,Threshold,"[T5 + UDG, Single Model (Google Brain)]"
2,Condorcet,[]
3,AM,[ERNIE 3.0]
4,GM,[ERNIE 3.0]
5,Minimax,"[ERNIE 3.0, SuperGLUE Human Baselines]"
6,Copeland,[SuperGLUE Human Baselines]
7,Dowdall,[SuperGLUE Human Baselines]
8,Baldwin,"[SuperGLUE Human Baselines, ERNIE 3.0]"
9,Borda,[ERNIE 3.0]


## VALUE

In [14]:
from votenrank.data_processing import preprocess_value

value = preprocess_value(pd.read_csv("../tables/leaderboards - VALUE.csv"))

### Results over missing scores

In [15]:
value_lb_with_nan = Leaderboard(value)
value_lb_with_nan_rank = value_lb_with_nan.rank_all(drop_mean=True)
value_lb_with_nan_rank

,Minimax,Copeland
Ranking position,,
1,-0.00: Human,6.00: Human
2,-6.00: craig.starr,4.00: craig.starr
3,-10.00: DuKG,2.00: DuKG
4,-10.00: HERO 1,0.00: HERO 1
5,-11.00: HERO 2,-2.00: HERO 3
6,-11.00: HERO 3,-4.00: HERO 2
7,-11.00: HERO 4,-6.00: HERO 4


In [16]:
value_lb_with_nan.elect_all(drop_mean=True)

,method,winners
0,Minimax,[Human]
1,Copeland,[Human]
2,Condorcet,[Human]


### Results over non-missing scores

In [17]:
value_lb_no_nans = Leaderboard(value[1:])
value_lb_no_nans.rank_all()

,Plurality,AM,GM,Minimax,Copeland,Dowdall,Borda
Ranking position,,,,,,,
1,9.00: craig.starr,62.87: craig.starr,49.96: craig.starr,-0.00: craig.starr,5.00: craig.starr,10.00: craig.starr,53.00: craig.starr
2,1.00: DuKG,60.00: DuKG,46.30: DuKG,-10.00: DuKG,3.00: DuKG,5.17: DuKG,39.00: DuKG
3,1.00: HERO 1,57.58: HERO 1,44.12: HERO 1,-10.00: HERO 1,1.00: HERO 1,4.08: HERO 1,30.00: HERO 1
4,0.00: HERO 2,56.96: HERO 3,43.22: HERO 3,-11.00: HERO 2,-1.00: HERO 3,2.87: HERO 2,20.00: HERO 2
5,0.00: HERO 3,56.07: HERO 2,42.81: HERO 2,-11.00: HERO 3,-3.00: HERO 2,2.82: HERO 3,18.00: HERO 3
6,0.00: HERO 4,52.59: HERO 4,37.56: HERO 4,-11.00: HERO 4,-5.00: HERO 4,2.02: HERO 4,5.00: HERO 4


### Selecting the winner

In [18]:
value_lb_no_nans.elect_all()

,method,winners
0,Plurality,[craig.starr]
1,Threshold,[craig.starr]
2,Condorcet,[craig.starr]
3,AM,[craig.starr]
4,GM,[craig.starr]
5,Minimax,[craig.starr]
6,Copeland,[craig.starr]
7,Dowdall,[craig.starr]
8,Baldwin,[craig.starr]
9,Borda,[craig.starr]
